# Biomass Dashboard API Tests

In [4]:
import requests
from typing import List

In [5]:
# papermill parameters cell

stage = "dit"


In [6]:
def populate_bm_api_host(stage):
    if stage == "production":
        return "https://gdehvjde39.execute-api.us-west-2.amazonaws.com/v1"
    elif stage == "staging":
        return "https://edovdgv7c6.execute-api.us-west-2.amazonaws.com/v1"
    elif stage in ["main", "dit"]:
        return "https://jsvuwyxsjh.execute-api.us-west-2.amazonaws.com/v1"
    else:

        exit(1)


bm_api_host = populate_bm_api_host(stage)

def get_by_id(items: List[dict], _id: str):
  return next(filter(
      lambda x: x["id"] == _id, items), None)


def test_resource(resource_name: str, min_count: int, entity_id: str, 
    include_global: bool = False, validate_same_list_entity: bool = True):

  res = requests.get(f"{bm_api_host}/{resource_name}")

  assert res.status_code == 200
  assert len(res.json()[resource_name]) >= min_count
  assert get_by_id(res.json()[resource_name], "non-existant") == None
  if include_global:
    assert get_by_id(res.json()[resource_name], "global")["id"] == "global"
  entity_from_list = get_by_id(res.json()[resource_name], entity_id)
  assert entity_from_list["id"] == entity_id

  res = requests.get(f"{bm_api_host}/{resource_name}/{entity_id}")
  assert res.status_code == 200

  if validate_same_list_entity:
    assert entity_from_list == res.json()


## Test Endpoints

In [7]:
test_resource("products", min_count=6, entity_id="cci_biomass", include_global=True)
test_resource("country_pilots", min_count=5, entity_id="japan")

# SKIP: https://github.com/MAAP-Project/biomass-dashboard-api/pull/16
# validate_same_list_entity can be removed when this bug gets fixed
test_resource("datasets", min_count=5, entity_id="icesat2_boreal_landsat8", validate_same_list_entity=False)
